In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.0886818895575825,0.46176741526610465,0.6845556…
  RigidBodyDynamics.Joint… => [0.7137174587310784]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [17]:
cache = MechanismStateCache(mechanism, x);

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{RigidBodyDynamics.SpatialInertia{T}}, ::RigidBodyDynamics.CartesianFrame3D, ::FixedSizeArrays.Vec{3,Float64}, ::Float64)
This may have arisen from a call to the constructor RigidBodyDynamics.SpatialInertia{T}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  RigidBodyDynamics.SpatialInertia{T}(::RigidBodyDynamics.CartesianFrame3D, !Matched::FixedSizeArrays.Mat{3,3,T}, !Matched::FixedSizeArrays.Vec{3,T}, !Matched::T)
  call{T}(::Type{T}, ::Any)
  convert{T}(::Type{T}, !Matched::T)
while loading In[17], in expression starting on line 1

In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

LoadError: LoadError: UndefVarError: cache not defined
while loading In[10], in expression starting on line 1

In [11]:
center_of_mass(mechanism, cache)

LoadError: LoadError: UndefVarError: cache not defined
while loading In[11], in expression starting on line 1

In [12]:
mass(mechanism)

1.1953160879466334

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

LoadError: LoadError: UndefVarError: cache not defined
while loading In[13], in expression starting on line 1

In [14]:
relative_twist(cache, body2, body1)

LoadError: LoadError: UndefVarError: cache not defined
while loading In[14], in expression starting on line 1

In [15]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

LoadError: LoadError: UndefVarError: cache not defined
while loading In[15], in expression starting on line 1

In [16]:
geometric_jacobian(cache, path(mechanism, body2, world))

LoadError: LoadError: UndefVarError: cache not defined
while loading In[16], in expression starting on line 1